In [2]:
import os
import shutil
import cv2
import numpy as np
from tqdm import tqdm
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import torch
from skimage.measure import label
from skimage.morphology import remove_small_objects

# ========== FOLDER SETUP ==========
WORK_DIR = r"C:\Users\ROG\Documents\Termatics\segmentation\detectron_maskrcnn\training_dataset_generated\training_sets"
ORTHO_PATH = r'ortho/solar_pv_True_Ortho.tif'
MODEL_WEIGHTS = os.path.join(WORK_DIR, "model", "best_model.pth")

TILE_DIR = os.path.join(WORK_DIR, "inference_tiles")
os.makedirs(TILE_DIR, exist_ok=True)

MERGED_MASK_PATH = os.path.join(WORK_DIR, "merged_mask.tif")
MERGED_VISUAL_PATH = os.path.join(WORK_DIR, "merged_visual_overlay.png")

# ========== STEP 1: TILE SETUP ==========
tile_size = 512
overlap = 64
image = cv2.imread(ORTHO_PATH)
if image is None:
    raise FileNotFoundError(f"Orthophoto not found at: {ORTHO_PATH}")
H, W, _ = image.shape

tile_coords = []
tile_id = 0
for y in tqdm(range(0, H, tile_size - overlap), desc="Splitting tiles"):
    for x in range(0, W, tile_size - overlap):
        x1, y1 = x, y
        x2, y2 = min(x + tile_size, W), min(y + tile_size, H)
        tile = image[y1:y2, x1:x2]
        tile_name = f"tile_{tile_id}.jpg"
        cv2.imwrite(os.path.join(TILE_DIR, tile_name), tile)
        tile_coords.append((tile_name, x1, y1, x2 - x1, y2 - y1))
        tile_id += 1

# ========== STEP 2: SETUP MODEL ==========
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = MODEL_WEIGHTS
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
predictor = DefaultPredictor(cfg)

# ========== STEP 3: RUN INFERENCE ON TILES AND MERGE ==========
global_mask = np.zeros((H, W), dtype=np.uint8)

for tile_name, x1, y1, w, h in tqdm(tile_coords, desc="Running inference"): 
    tile_img = cv2.imread(os.path.join(TILE_DIR, tile_name))
    outputs = predictor(tile_img)
    instances = outputs["instances"].to("cpu")
    masks = instances.pred_masks.numpy()

    crop_margin = overlap // 2
    crop_x1 = crop_margin if x1 > 0 else 0
    crop_y1 = crop_margin if y1 > 0 else 0
    crop_x2 = w - crop_margin if x1 + tile_size < W else w
    crop_y2 = h - crop_margin if y1 + tile_size < H else h

    for mask in masks:
        # vegetation filter
        masked_area = tile_img[mask]
        if masked_area.size == 0:
            continue
        avg_color = np.mean(masked_area, axis=0)
        if avg_color[1] > avg_color[0] + 15 and avg_color[1] > avg_color[2] + 15:
            continue

        cropped_mask = mask[crop_y1:crop_y2, crop_x1:crop_x2]
        px1 = x1 + crop_x1
        py1 = y1 + crop_y1
        px2 = px1 + (crop_x2 - crop_x1)
        py2 = py1 + (crop_y2 - crop_y1)

        global_mask[py1:py2, px1:px2] |= cropped_mask.astype(np.uint8)

# ========== STEP 4: REMOVE SMALL OBJECTS AND SAVE ==========
labeled_mask = label(global_mask.astype(bool))
cleaned_mask = remove_small_objects(labeled_mask, min_size=500)

# Convert to binary mask (0 and 255)
binary_mask = (cleaned_mask > 0).astype(np.uint8) * 255

# Save binary mask
cv2.imwrite(MERGED_MASK_PATH, binary_mask)

# Optional: Visual overlay
visual_overlay = image.copy()
visual_overlay[cleaned_mask > 0] = (0.7 * image[cleaned_mask > 0] + 0.3 * np.array([0, 255, 0])).astype(np.uint8)
cv2.imwrite(MERGED_VISUAL_PATH, visual_overlay)
print(f"Merged binary mask saved to: {MERGED_MASK_PATH}")
print(f"Merged visual overlay saved to: {MERGED_VISUAL_PATH}")

# Cleanup
shutil.rmtree(TILE_DIR)
print(f"Deleted temporary tile directory: {TILE_DIR}")


Running inference:   0%|                                                                      | 0/4400 [00:00<?, ?it/s]C:\Users\ROG\.conda\envs\detectron_env\lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Running inference: 100%|███████████████████████████████████████████████████████████| 4400/4400 [05:29<00:00, 13.34it/s]


Merged binary mask saved to: C:\Users\ROG\Documents\Termatics\segmentation\detectron_maskrcnn\training_dataset_generated\training_sets\merged_mask.tif
Merged visual overlay saved to: C:\Users\ROG\Documents\Termatics\segmentation\detectron_maskrcnn\training_dataset_generated\training_sets\merged_visual_overlay.png
Deleted temporary tile directory: C:\Users\ROG\Documents\Termatics\segmentation\detectron_maskrcnn\training_dataset_generated\training_sets\inference_tiles


add id to solar panels

In [ ]:
#Extract Polygons from masks

In [3]:
import os
import cv2
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
from rasterio.features import shapes
from rasterio.transform import Affine

# === PATHS ===
ORTHO_PATH = r"ortho/solar_pv_True_Ortho.tif"
MASK_PATH = r"C:\Users\ROG\Documents\Termatics\segmentation\detectron_maskrcnn\training_dataset_generated\training_sets\merged_mask.tif"
SAVE_PATH = r"C:\Users\ROG\Documents\Termatics\segmentation\detectron_maskrcnn\training_dataset_generated\training_sets\solar_polygons.geojson"  # or .shp

# === LOAD ORTHO TO GET CRS AND TRANSFORM ===
with rasterio.open(ORTHO_PATH) as src:
    transform: Affine = src.transform
    crs = src.crs

# === LOAD BINARY MASK ===
mask = cv2.imread(MASK_PATH, cv2.IMREAD_GRAYSCALE)
if mask is None:
    raise FileNotFoundError(f"Mask not found: {MASK_PATH}")

# Ensure binary
_, binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
binary = (binary > 0).astype(np.uint8)

# === EXTRACT POLYGONS WITH GEOMETRY ===
shapes_gen = shapes(binary, mask=binary, transform=transform)

geoms = []
ids = []
for i, (geom, val) in enumerate(shapes_gen):
    if val == 1:
        poly = Polygon(geom["coordinates"][0])
        if poly.is_valid and poly.area > 1.0:  # remove noise
            geoms.append(poly)
            ids.append(i + 1)

# === SAVE TO GEOFILE ===
gdf = gpd.GeoDataFrame({"id": ids}, geometry=geoms, crs=crs)
gdf.to_file(SAVE_PATH, driver="GeoJSON")  # change to "ESRI Shapefile" if .shp

print(f"✓ Polygon file saved at: {SAVE_PATH}")


✓ Polygon file saved at: C:\Users\ROG\Documents\Termatics\segmentation\detectron_maskrcnn\training_dataset_generated\training_sets\solar_polygons.geojson
